In [20]:
import cv2
import easyocr
import numpy as np
import pandas as pd
import torch
import os
import urllib.request

from fuzzywuzzy import process
from pdf2image import convert_from_path
from PIL import Image

from webscraping import extract_all_pdfs
from text_extraction import process_image, display_opencv_image

In [21]:
print(f"CUDA Available: {torch.cuda.is_available()}")

# Check if CuDNN is enabled in PyTorch
print(f"CuDNN Enabled: {torch.backends.cudnn.enabled}")

# Check CuDNN version in PyTorch
print(f"CuDNN Version: {torch.backends.cudnn.version()}")

CUDA Available: True
CuDNN Enabled: True
CuDNN Version: 90100


# Link and File Paths

In [22]:
ODCY_LINK = "https://childcaresearch.ohio.gov/search?q=fVLNbhMxEN40v0uapgKhHhAiBy6VQkUR1xwWN1VDIVl1V0gFcXDWk42FY6%2b83pS98Q6ICxdeg1fgyBvwJjB2uhCJqrPS2DOf59tvPPZqnuf9RrOrtd0ddCRhyRXMiVqtlBwO3oDOuZKj46On9hsOSCFMoWEkoTCaiuEgLOaCJ%2bdQxuoDyJEshGhaxkfbREcn8ewoAqqTJdHcgOb0MZ7ZC7VacwZ6WqzmoFtEFdKUDcJN2X7LM6IY7FZH4jKD1gU1XKbdkGrDqZjSFbQnckGlyXuXWJvGijGBknszYUuuIz%2fUkCdLpYQfuSVIoX9Cy3y2mGWgkVPJ%2fTNV6O1E8zVQke%2b%2fgIXSsCkjVEN3vAaJGuy%2bE13x1QqD%2b1EGiRUEwHKy5IJZeC%2fWVOaZ0sYR9oMFNv6PqTdbg5Y8XRob3TnlIFiseZYfjKkWpaPBs2z8MUMaJPDPgLLIYO%2f3Qs3X1MA5l9hmihmQB5tBiPK0wCT7q%2bLuK07nXOCVTmReoKIEmtNgfEma0zEJA9wTEqI%2fJaROZkEjINEEB5FKThsThLxaDce5eSPthnejNTudmrOH%2fw3dXesFlSm8e2%2ffV%2b1Gs8jOk%2bNW3bL59cpVL9MFm51V4DdvloHWt3h7q7TtztrIIo0Htwi0IvyWdZbA71jn2%2b5s7rZCp9cNJub4IseS2bUTgYDEAHM8rkUv%2b%2frymV1%2fPf%2fCnC77p9Z25vvn4JtDOhVynfl0%2bPPwh0P8CqmsYu3%2bAQ%3d%3d"

REL_PATH = "https://childcaresearch.ohio.gov/"

# Helper Functions

In [23]:
def group_into_rows(extracted_text, threshold=5):
    """
    Group extracted_text entries into rows based on their y-coordinates.
    
    :param extracted_text: A list of tuples of the form:
                          [ ( (x, y, w, h), [ (text, conf), ... ] ), ... ] 
                          assumed to be sorted top-to-bottom, left-to-right.
    :param threshold:    The distance in pixels to decide when to start a new row.
    :return:               A list (rows) of lists, each inner list is one row, 
                           containing the sub-rectangle data.
    """
    rows = []
    current_row = []
    if not extracted_text:
        return rows

    # start the first row’s baseline from the very first rectangle's y
    _, first_data = extracted_text[0]
    current_row_y = extracted_text[0][0][1]
    current_row_h = extracted_text[0][0][3]

    for ((x, y, w, h), text_data) in extracted_text:

        if abs(y - current_row_y) > threshold or abs(h - current_row_h) > threshold:
            # push the old row into rows
            rows.append(current_row)
            # start a new row
            current_row = []
            current_row_y = y
            current_row_h = h


        # add current bounding box/data to the current row
        current_row.append(((x, y, w, h), text_data))

    if current_row:
        rows.append(current_row)

    # sort each row by x-coordinate and remove empty rows
    rows = [sorted(r, key=lambda x: x[0][0]) for r in rows if not (len(r) == 1 and len(r[0][1]) == 0)]

    return rows

In [24]:
def process_ocr_to_dataframe(ocr_results, fields):
    """
    Process OCR results and flatten the extracted information into a CSV-ready format.

    Parameters:
    ocr_results (dict): Dictionary of OCR results containing bounding boxes and text.

    Returns:
    pd.DataFrame: A dataframe containing flattened, structured data for CSV storage.
    """
    extracted_data = {field: None for field in fields}

    for field in ocr_results:
        if len(field[1]) > 1:

            label, content = field[1][0][0], field[1][-1][0]
            confidence = field[1][0][1]

            best_match, score = process.extractOne(label, extracted_data.keys())
            if score > 95:
                if extracted_data[best_match] is not None and extracted_data[best_match] != content:
                    print(f"Field '{best_match}' field changed from '{extracted_data[best_match]}' to '{content}'")

                extracted_data[best_match] = content, confidence

    general_df = pd.DataFrame([extracted_data])

    general_df["Program Name"] = general_df.apply(lambda x: x["Program Name"][0], axis=1)
    general_df.set_index("Program Name", inplace=True)

    # general_df = general_df[sorted(general_df.columns)]

    return general_df

In [25]:
def get_row(rows, field, start_idx=0):
    idx = start_idx
    i = start_idx

    while i < len(rows) and idx == start_idx:
        row = rows[i][0][1][0][0]
        if field in row or row in field:
            idx = i

        # else: 
        # print(f"Field: {field} and Row: {row}")

        i += 1

    if idx == start_idx:
        print(f"Field '{field}' not found in rows.")

    return idx

def extract_license_capacity_table(extracted_text):
    rows = group_into_rows(extracted_text)

    table_rows = [
        "Infant",
        "Young Toddler",
        "Total Under 2",
        "Older Toddler",
        "Preschool",
        "School",
        "Total Capacity/Enrollment"]

    columns = [
        "Full Time",
        "Part Time",
        "Total"
    ]

    table = {"License Capacity": {}}

    row_idx = 0

    for i, t_row in enumerate(table_rows):

        prev = row_idx
        row_idx = get_row(rows, t_row, row_idx)

        if row_idx == prev:
            continue

        current_row = rows[row_idx]

        # save license capacity totals
        if len(current_row) > len(columns) + 1:
            table["License Capacity"][t_row] = current_row[1][1]
            current_row = current_row[2:]
        else:
            current_row = current_row[1:]

        table[t_row] = {columns[i]: field[1][0] for i, field in enumerate(current_row)}

    return table

# Extract PDF Links

In [26]:
test_link = "https://childcaresearch.ohio.gov/search?q=fVLNbhMxEN40v0uapgKhHhAiBy6VQkUR1xwWN1VDIVl1V0gFcXDWk42FY6%2b83pS98Q6ICxdeg1fgyBvwJjB2uhCJqrPS2DOf59tvPPZqnuf9RrOrtd0ddCRhyRXMiVqtlBwO3oDOuZKj46On9hsOSCFMoWEkoTCaiuEgLOaCJ%2bdQxuoDyJEshGhaxkfbREcn8ewoAqqTJdHcgOb0MZ7ZC7VacwZ6WqzmoFtEFdKUDcJN2X7LM6IY7FZH4jKD1gU1XKbdkGrDqZjSFbQnckGlyXuXWJvGijGBknszYUuuIz%2fUkCdLpYQfuSVIoX9Cy3y2mGWgkVPJ%2fTNV6O1E8zVQke%2b%2fgIXSsCkjVEN3vAaJGuy%2bE13x1QqD%2b1EGiRUEwHKy5IJZeC%2fWVOaZ0sYR9oMFNv6PqTdbg5Y8XRob3TnlIFiseZYfjKkWpaPBs2z8MUMaJPDPgLLIYO%2f3Qs3X1MA5l9hmihmQB5tBiPK0wCT7q%2bLuK07nXOCVTmReoKIEmtNgfEma0zEJA9wTEqI%2fJaROZkEjINEEB5FKThsThLxaDce5eSPthnejNTudmrOH%2fw3dXesFlSm8e2%2ffV%2b1Gs8jOk%2bNW3bL59cpVL9MFm51V4DdvloHWt3h7q7TtztrIIo0Htwi0IvyWdZbA71jn2%2b5s7rZCp9cNJub4IseS2bUTgYDEAHM8rkUv%2b%2frymV1%2fPf%2fCnC77p9Z25vvn4JtDOhVynfl0%2bPPwh0P8CqmsYu3%2bAQ%3d%3d&p=1"

In [27]:
pdf_link_path = "pdf_links.csv"

if os.path.exists(pdf_link_path):
    pdf_links = pd.read_csv(pdf_link_path, index_col=0)
else:
    pdf_links = extract_all_pdfs(test_link, REL_PATH)
    pdf_links.to_csv(pdf_link_path, index=True)
pdf_links

,pdf,Address,City,Zip
program_name,,,,
A BRIGHT START 4 KIDZ LEARNING CTR,https://childcaresearch.ohio.gov//pdf/00224002...,8211 PLATT,CLEVELAND,44104
A BRIGHTER START CHILDCARE,https://childcaresearch.ohio.gov//pdf/00000020...,2765 BLUE ROCK RD.,CINCINNATI,45239
A CHILD'S GARDEN,https://childcaresearch.ohio.gov//pdf/00000020...,5427 JULMAR DRIVE,CINCINNATI,45238
A CHILD'S JOURNEY LEARNING CENTER,https://childcaresearch.ohio.gov//pdf/00217001...,846 S. YEARLING RD,WHITEHALL,43213
A CHILD'S PLACE LEARNING CENTER,https://childcaresearch.ohio.gov//pdf/00000040...,2010 OFFICEVIEW PLACE,REYNOLDSBURG,43068
A GREAT START PRESCHOOL INC,https://childcaresearch.ohio.gov//pdf/00000020...,7001 FAR HILLS AVE,DAYTON,45459
A JOYFUL JOURNEY ACADEMY,https://childcaresearch.ohio.gov//pdf/00222002...,1536 BARNETT ROAD,COLUMBUS,43227
A JUBILEE ACADEMY,https://childcaresearch.ohio.gov//pdf/00000030...,15751 LAKESHORE BLVD,CLEVELAND,44110
A KIDS ONLY EARLY LEARNING CENTER INC. 4,https://childcaresearch.ohio.gov//pdf/00219001...,2505 SOUTH RIDGE EAST,ASHTABULA,44004


In [28]:
local_file, _ = urllib.request.urlretrieve(pdf_links.iloc[1]['pdf'])
local_file

'C:\\Users\\WILLBL~1\\AppData\\Local\\Temp\\tmpkgyhbftc'

# Extract Text from PDF

In [29]:
images = convert_from_path(local_file, dpi=500)

In [30]:
image = images[0]

In [41]:
len(images)

12

In [31]:
ocr = easyocr.Reader(['en'], gpu=True)

In [32]:
extracted_text = process_image(image, ocr, verbose=True, display=True)
extracted_text

Detected 72 hierarchical sub-rectangles.
No text detected in sub-rectangle 6.
Extracted 71 / 72 text fields with an average confidence of 0.92.


[((254, 1261, 3744, 99), [('Program Details', np.float64(0.95))]),
 ((253, 1365, 1429, 280),
  [('Program Name', np.float64(0.82)),
   ('A BRIGHTER START CHILDCARE', np.float64(0.99))]),
 ((1686, 1365, 1340, 280),
  [('Program Number', np.float64(0.94)), ('000000200979', np.float64(1.0))]),
 ((3030, 1365, 969, 280),
  [('Program Type', np.float64(1.0)), ('Child Care Center', np.float64(0.9))]),
 ((253, 1648, 2773, 374),
  [('Address', np.float64(1.0)),
   ('2765 BLUE ROCK RD. CINCINNATI', np.float64(0.95)),
   ('OH', np.float64(1.0)),
   ('45239', np.float64(1.0))]),
 ((3030, 1648, 969, 374),
  [('County', np.float64(1.0)), ('HAMILTON', np.float64(1.0))]),
 ((254, 2026, 3744, 91), []),
 ((253, 2122, 1429, 186), [('Building Approval Date', np.float64(0.81))]),
 ((1686, 2122, 642, 186), [('Use Group/Code', np.float64(0.95))]),
 ((2332, 2122, 829, 186), [('Occupancy Limit', np.float64(1.0))]),
 ((3165, 2122, 834, 186), [('Maximum Under 2 Y', np.float64(0.74))]),
 ((253, 2312, 1429, 186),


In [33]:
p1_fields = [ "Program Name",
        "Program Number",
        "Program Type",
        "County",
        "Building Approval Date",
        "Use Group/Code",
        "Occupancy Limit",
        "Maximum Under 2",   # under 2 1/2 but idk how this will be read...
        "Fire Inspection Approval Date",
        "Food Service Risk Level",
        "Inspection Type",
        "Inspection Scope",
        "Inspection Notice",
        "Inspection Date",
        "Begin Time",
        "End Time",
        "Reviewer",
        "No. Rules Verified",
        "No. Rules with Non-compliances",
        "No. Serious Risk",
        "No. Moderate Risk",
        "No. Low Risk"
]

In [34]:
process_ocr_to_dataframe(extracted_text, p1_fields)

,Program Number,Program Type,County,Building Approval Date,Use Group/Code,Occupancy Limit,Maximum Under 2,Fire Inspection Approval Date,Food Service Risk Level,Inspection Type,...,Inspection Notice,Inspection Date,Begin Time,End Time,Reviewer,No. Rules Verified,No. Rules with Non-compliances,No. Serious Risk,No. Moderate Risk,No. Low Risk
Program Name,,,,,,,,,,,,,,,,,,,,,
A BRIGHTER START CHILDCARE,"(000000200979, 0.94)","(Child Care Center, 1.0)","(HAMILTON, 1.0)",None,None,None,None,"(09/20/2024, 0.93)","(Level II, 0.5)","(Annual, 1.0)",...,"(Unannounced, 1.0)","(10/02/2024, 1.0)","(9:00 AM, 0.88)","(11.30 AM, 0.93)","(Kristin Blassingame, 1.0)","(58, 0.6)","(10, 0.76)",None,"(2, 0.98)","(9, 0.98)"


In [35]:
extract_license_capacity_table(extracted_text)

{'License Capacity': {'Total Under 2': [('21', np.float64(1.0))],
  'Total Capacity/Enrollment': [('44', np.float64(1.0))]},
 'Infant': {'Full Time': ('11', np.float64(1.0)),
  'Part Time': ('0', np.float64(1.0)),
  'Total': ('11', np.float64(1.0))},
 'Young Toddler': {'Full Time': ('0', np.float64(0.85)),
  'Part Time': ('0', np.float64(1.0)),
  'Total': ('0', np.float64(1.0))},
 'Total Under 2': {'Full Time': ('11', np.float64(1.0)),
  'Part Time': ('0', np.float64(1.0)),
  'Total': ('11', np.float64(1.0))},
 'Older Toddler': {'Full Time': ('0', np.float64(1.0)),
  'Part Time': ('0', np.float64(1.0)),
  'Total': ('0', np.float64(0.77))},
 'Preschool': {'Full Time': ('21', np.float64(1.0)),
  'Part Time': ('0', np.float64(0.65)),
  'Total': ('21', np.float64(1.0))},
 'School': {'Full Time': ('0', np.float64(1.0)),
  'Part Time': ('0', np.float64(1.0)),
  'Total': ('0', np.float64(1.0))},
 'Total Capacity/Enrollment': {'Full Time': ('21', np.float64(1.0)),
  'Part Time': ('0', np.float

In [36]:
rows = group_into_rows(extracted_text)
rows[15]

[((253, 4044, 932, 93),
  [('Infant', np.float64(1.0)), ('Birth to < 18 m)', np.float64(0.73))]),
 ((1783, 4044, 372, 93), [('11', np.float64(1.0))]),
 ((2158, 4044, 661, 93), [('0', np.float64(1.0))]),
 ((2822, 4044, 1177, 93), [('11', np.float64(1.0))])]

In [39]:
page_2 = process_image(images[1], ocr, display=True)
group_into_rows(page_2)

[[((253, 544, 1181, 94), [('Infant/Toddler', np.float64(1.0))]),
  ((1437, 544, 1183, 94), [('0 to < 12 months', np.float64(0.76))]),
  ((2623, 544, 732, 94), [('1to 5', np.float64(0.94))]),
  ((3359, 544, 638, 94), [])],
 [((253, 641, 1181, 94), [('Infant/ Toddler', np.float64(0.71))]),
  ((1437, 641, 1183, 94), [('0 to < 12 months', np.float64(0.9))]),
  ((2623, 641, 732, 94), [('1to 4', np.float64(0.97))]),
  ((3359, 641, 638, 94), [])],
 [((253, 738, 1181, 373), [('preschool', np.float64(1.0))]),
  ((1437, 738, 1183, 373),
   [('3 years to', np.float64(0.97)), ('4 years', np.float64(0.97))]),
  ((2623, 738, 732, 373), []),
  ((3359, 738, 638, 373),
   [("Preschool and 4's", np.float64(0.92)),
    ('were combined at', np.float64(0.83)),
    ('the time ratio', np.float64(0.99)),
    ('was', np.float64(1.0)),
    ('taken.', np.float64(0.81))])],
 [((253, 1115, 1181, 373), [('preschool', np.float64(1.0))]),
  ((1437, 1115, 1183, 373), [('3 years to < 4 years', np.float64(0.91))]),
  ((

In [40]:
page_3 = process_image(images[2], ocr, display=True)
group_into_rows(page_3)

[[((253, 544, 3744, 4528),
   [('4. Child care staff were using a baby monitor to supervise children.',
     np.float64(0.72)),
    ('5. Child care staff were using a walkie talkie to supervise children:',
     np.float64(0.7)),
    ('6. Child care staff were', np.float64(0.76)),
    ('mirrors to view children in another room:', np.float64(0.81)),
    ('7. Child care staff were using a video camera instead of physically being present in the room.',
     np.float64(0.58)),
    ('8. Other [', np.float64(1.0)),
    ('].', np.float64(0.32)),
    ('Children must be supervised and within sight and hearing of a child care staff member at all times. Provide staff',
     np.float64(0.59)),
    ("training: Submit the program'$ corrective action plan, which includes a statement that training was provided, to",
     np.float64(0.54)),
    ('the Department to verify compliance with the requirements of this rule.',
     np.float64(0.75)),
    ('Corrective Action Plan Due: 11/01/2024', np.float64(0.4